# **Importing Libraries**

In [1]:
import tensorflow as tf

In [ ]:
print("TensorFlow Version:", tf.__version__)
print("Available physical devices:", tf.config.list_physical_devices())

In [12]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import mixed_precision
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import os

print("Libraries imported successfully.")

Libraries imported successfully.


In [ ]:
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'

# mixed_precision.set_global_policy('mixed_float16')
 
# print("Environment set up for ANE")

# **Data Paths**

In [ ]:
train_path = 'Beef Photo Background Remove/TRAIN/'
test_path = 'Beef Photo Background Remove/TEST/'
batch_size = 16

print(f"Train path: {train_path}")
print(f"Test path: {test_path}")
print(f"Batch size: {batch_size}")

# **Image resize**

In [6]:
def load_image(file_path, label):
    img = tf.io.read_file(file_path)
    img = tf.image.decode_image(img, channels=3, expand_animations=False)

    img.set_shape([None, None, 3])
    
    img = tf.image.resize(img, (800, 800))

    img = tf.image.convert_image_dtype(img, tf.float32)

    return img, label

In [8]:
from PIL import Image

def check_image_files(directory):
    for root, _, files in os.walk(directory):
        for file_name in files:
            file_path = os.path.join(root, file_name)
            try:
                with Image.open(file_path) as img:
                    img.verify()
            except (IOError, SyntaxError) as e:
                print(f"Corrupted or invalid image file detected: {file_path}")

check_image_files(train_path)
check_image_files(test_path)

# **Create a Function to Generate Dataset from Directory**

In [ ]:
def get_dataset_from_directory(directory, batch_size=32, shuffle=True):
    # Get class names and create label mapping
    class_names = [d for d in os.listdir(directory) if os.path.isdir(os.path.join(directory, d))]
    class_indices = {name: index for index, name in enumerate(class_names)}
    
    # Get all image file paths and labels
    file_paths = []
    labels = []
    for class_name in class_names:
        class_dir = os.path.join(directory, class_name)
        # Skip non-directory files
        for file_name in os.listdir(class_dir):
            file_path = os.path.join(class_dir, file_name)
            if os.path.isfile(file_path):  # Make sure it's a file, not a directory
                file_paths.append(file_path)
                labels.append(class_indices[class_name])
    
    # Convert to TensorFlow dataset
    file_paths_ds = tf.data.Dataset.from_tensor_slices(file_paths)
    labels_ds = tf.data.Dataset.from_tensor_slices(labels)
    dataset = tf.data.Dataset.zip((file_paths_ds, labels_ds))
    
    # Map the load_image function
    dataset = dataset.map(load_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    
    # Shuffle, batch, and prefetch
    if shuffle:
        dataset = dataset.shuffle(buffer_size=len(file_paths))
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    
    return dataset, class_indices

print("Dataset loading function defined.")

# **Load Training and Testing Datasets**

In [ ]:
train_dataset, class_indices = get_dataset_from_directory(train_path, batch_size=batch_size, shuffle=True)
test_dataset, _ = get_dataset_from_directory(test_path, batch_size=batch_size, shuffle=False)

print(f"Classes found: {class_indices}")

# **Update the Model Input to Handle Variable Sizes**

In [ ]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(800, 800, 3))

for layer in base_model.layers[-100:]:
    layer.trainable = True

x = GlobalAveragePooling2D()(base_model.output)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu', kernel_regularizer=l2(0.01))(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = Dense(128, activation='relu', kernel_regularizer=l2(0.01))(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
predictions = Dense(len(class_indices), activation='softmax', dtype='float32')(x)

model = Model(inputs=base_model.input, outputs=predictions)

print("Model architecture built.")

In [ ]:
model.compile(optimizer=Adam(learning_rate=3e-4),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

print("Model compiled successfully.")

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=1e-7, verbose=1)

print("Callbacks for training defined.")

# **Train the Model**

In [ ]:
history = model.fit(
    train_dataset,
    epochs=5,
    validation_data=test_dataset,
    callbacks=[early_stopping, reduce_lr],
    verbose=1
)

print("Model training completed.")

In [ ]:
model.save('Beef_Classification.h5')

# **Model Evaluation**

In [ ]:
test_loss, test_acc = model.evaluate(test_dataset)
print(f"Test Accuracy: {test_acc:.4f}")

predictions = model.predict(test_dataset)
y_pred = np.argmax(predictions, axis=1)
y_true = np.concatenate([y for x, y in test_dataset], axis=0)

print(classification_report(y_true, y_pred, target_names=class_indices.keys()))

# **Confusion Matrix**

In [ ]:
conf_matrix = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues",
            xticklabels=class_indices.keys(),
            yticklabels=class_indices.keys())
plt.xlabel("Predicted")
plt.ylabel("True")
plt.show()